# Problem 2

(2) In class we have seen implementations of the mid-point rule and Simpson's 3/8-method to approximate integrals. One of the key features of these methods (especially useful when your function comes from measured data) is that the technique uses as its input **only** the values of $f(x)$ at a finite number of points.  Sometimes one has access to **more** information about $f$ than simply its values.  In this problem, we ask you to write a script to approximate integrals numerically, but where one has a **symbolic** expression for $f$, and therefore using sympy, one also has access to $f'$. 

**Fact** Given an interval $[a,b]$ and numbers $f_a, f'_a, f_b, f'_b$ there exists one (and only one) cubic polynomial $p(x) = c_0 + c_1 x + c_2 x^2 + c_3 x^3$ such that $$p(a) = f_a, \ \ p'(a) = f'_a, \ \ p(b) = f_b, \ \ p'(b) = f'_b$$

Moreover, one can solve for $c_0,c_1,c_2,c_3$ fairly easily.  The above equations are:

$$c_0 + c_1 a + c_2 a^2 + c_3 a^3 = f_a \ [eq.1]\ \ c_0 + c_1 b + c_2 b^2 + c_3 b^3 = f_b \ [eq.2] $$
$$c_1 + 2c_2a + 3c_3a^2 = f'_a \ [eq.3] \ \ c_1 + 2c_2b+3c_3b^2 = f'_b \ [eq.4] $$

a little manipulation and one gets to the expression:

$$\pmatrix{ c_2 \\ c_3} = \frac{1}{(a-b)^3} \cdot \pmatrix{ 3(b+a) & -2(b^2+ba+a^2) \\ -2 & b+a} \cdot \pmatrix{ f_0 - f_1 + bf_1' - af_0' \\ f_1' -f_0'}$$

the middle expression is a $2 \times 2$ matrix. From the above one can solve for $c_0$ and $c_1$ by back-substituting into [eq.1] and [eq.3] (or [eq.2] and [eq.4], your choice). 

Compare the accuracy of this method to the mid-point and Simpson 3/8-method, for the computation of the integral $\int_0^\pi \sin(x) dx = 2$. Try the number of intervals $n=10, 100, 1000$ for all three methods. 

Aside from accuracy, what other considerations would cause you to prefer one method over the other two?

In [40]:
import sympy
import mpmath
import numpy
from IPython.display import display

sympy.init_printing(use_latex='mathjax')



def midpoint_integral(f, I, k): 
    sum = 0
    deltax = (I[1]-I[0])/float(k)
    for i in range (0, k):
        x = ((i/float(k))*I[1]) + ((1.0-(i/float(k)))*I[0]) + deltax/2
        sum = sum + f(x)*deltax
    return sum


def three_eights_integral(func, I, subdivisions): 
    sum = 0
    deltax = (I[1]-I[0]) / subdivisions
    for i in range (0, subdivisions):
        xa = ((i / float(subdivisions)) *I[1]) + ((1-i / float(subdivisions)) *I[0]) 
        xb = ((i / float(subdivisions)) *I[1]) + ((1-i / float(subdivisions)) *I[0]) + deltax / 3 
        xc = ((i / float(subdivisions)) *I[1]) + ((1-i / float(subdivisions)) *I[0]) + (2 * deltax) / 3
        xd = ((i / float(subdivisions)) *I[1]) + ((1-i / float(subdivisions)) *I[0]) + deltax
        sum = sum + (func(xa) + 3*func(xb) + 3*func(xc) + func(xd)) * deltax / 8
    return sum

def described_method(function, interval, subdivisions):
    x, c_0, c_1, c_2, c_3 = sympy.symbols("x, c_0, c_1, c_2, c_3")
    f_a, f_b = sympy.symbols("f_a, f_b")
    #.subs(c_3, calc_c_3).subs(c_2, calc_c_2)
    
    total = 0
    
    deltax = (interval[1] - interval[0]) / float(subdivisions)
    for index in range(0, subdivisions-1):
        a = deltax*index
        b = deltax*(index + 1)
        f_a_calc = function(a)
        fprime_a_calc = sympy.diff(function(x)).evalf(subs= { x: a })
        f_b_calc = function(b)
        fprime_b_calc = sympy.diff(function(x)).evalf(subs= { x: b })
        # Do the work
        calc_c_2, calc_c_3 = described_function(a, b, f_a_calc, fprime_a_calc, f_b_calc, fprime_b_calc)
        # Calc
        calc_c_1 = sympy.solve(
            sympy.Eq(c_1 + (2 * c_2 * a) + (3 * c_3 * (a**2)), f_a_calc)
                .subs(c_2, calc_c_2)
                .subs(c_3, calc_c_3),
            c_1)[0]
        calc_c_0 = sympy.solve(
            sympy.Eq(c_0 + (c_1 * a) + (c_2 * (a**2)) + (c_3 * (a**3)), fprime_a_calc)
                .subs(c_1, calc_c_1)
                .subs(c_2, calc_c_2)
                .subs(c_3, calc_c_3),
            c_0)[0]
        
        calc_value_a = (c_0 + (c_1 * x) + (c_2 * (x**2)) + (c_3 * (x**3))).evalf(subs= {
                c_0: calc_c_0,
                c_1: calc_c_1,
                c_2: calc_c_2,
                c_3: calc_c_3,
                x: a
            })
        calc_value_b = (c_0 + (c_1 * x) + (c_2 * (x**2)) + (c_3 * (x**3))).evalf(subs= {
                c_0: calc_c_0,
                c_1: calc_c_1,
                c_2: calc_c_2,
                c_3: calc_c_3,
                x: b
            })
        total += calc_value_b - calc_value_a
    return total
        
        
    
def described_function(a_in, b_in, f_a_in, fprime_a_in, f_b_in, fprime_b_in):
    a, b = sympy.symbols("a, b")
    f_a, fprime_a, f_b, fprime_b = sympy.symbols("f_a, f\prime_a, f_b, f\prime_b")
    # Define the left term.
    left_term = (1.0 / ((a - b) ** 3))
    # Define the middle term.
    mid_term = sympy.Matrix([
            [3 * (b + a), -2 * ((b ** 2) + (b*a) + (a ** 2))],
            [-2, b + a]
        ])
    # Define the right term.
    right_term = sympy.Matrix([
            f_a - f_b + (b * fprime_b) - (a * fprime_a),
            fprime_b - fprime_a
        ])

    return (left_term * sympy.Matrix(mid_term.dot(right_term))).evalf(subs = {
            a: a_in,
            b: b_in,
            f_a: f_a_in,
            fprime_a: fprime_a_in,
            f_b: f_b_in,
            fprime_b: fprime_b_in,

        })
    
    
    

midpoint_results = [print("Midpoint", subdivisions, midpoint_integral(numpy.sin, [0.0, numpy.pi], subdivisions))
                    for subdivisions in [10, 100, 1000]]


three_eights_results = [print("Three_eights", subdivisions, three_eights_integral(numpy.sin, [0.0, numpy.pi], int(subdivisions / 3)))
                        for subdivisions in [10, 100, 1000]]

described_results = [print("Described", subdivisions, described_method(sympy.sin, [0.0, numpy.pi], int(subdivisions)))
                        for subdivisions in [10]]

Midpoint 10 2.00824840791
Midpoint 100 2.00008224907
Midpoint 1000 2.00000082247
Three_eights 10 2.00038224209
Three_eights 100 2.00000002536
Three_eights 1000 2.0
Described 10 1.58251749811499


In [66]:
x = sympy.Symbol("x")
sympy.diff(sin(x))

cos(x)